In [1]:
import numpy as np 
import pandas as pd

In [65]:
df=pd.read_csv('/content/drive/MyDrive/nlp/datasets/IMDB_Dataset.csv')

In [66]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
data=pd.read_csv('/content/drive/MyDrive/nlp/datasets/IMDB_Dataset.csv')

In [3]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [64]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production br br The filmin...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically theres a family where a little boy J...,0
4,Petter Matteis Love in the Time of Money is a ...,1


In [5]:
data.isna().sum()

review       0
sentiment    0
dtype: int64

In [6]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
import re

In [8]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [9]:
data["review"] =data["review"].apply(lambda x:remove_between_square_brackets(x))

In [10]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
data['review']= data['review'].apply(lambda x: cleaning_punctuations(x))
data['review'].tail()

49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, dtype: object

In [11]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['review'] = data['review'].apply(lambda x: cleaning_numbers(x))
data['review'].tail()

49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, dtype: object

In [12]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

In [13]:

from sklearn.preprocessing import LabelBinarizer

In [14]:
lb=LabelBinarizer()
data['sentiment']=lb.fit_transform(data['sentiment'])

In [15]:
data_texts=data['review']
data_labels=data['sentiment']

In [16]:
data_texts

0        One of the other reviewers has mentioned that ...
1        A wonderful little production br br The filmin...
2        I thought this was a wonderful way to spend ti...
3        Basically theres a family where a little boy J...
4        Petter Matteis Love in the Time of Money is a ...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [17]:
data_labels

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
vectorizer = CountVectorizer(max_features=10,lowercase=False)
tf_idf_transformer = TfidfTransformer()

In [20]:
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(data_texts))
x_train_weight = tf_idf.toarray()

In [21]:
x_train_weight[1]

array([0.38341113, 0.48529811, 0.        , 0.17588958, 0.12833449,
       0.27733592, 0.        , 0.69326812, 0.        , 0.11197712])

In [22]:
from sklearn.model_selection import train_test_split

# Split Train and test data
train_sentiment, test_sentiment, train_review, test_review = train_test_split(x_train_weight, data_labels, test_size=0.2, random_state=0)

In [23]:

test_review

11841    1
19602    0
45519    0
25747    1
42642    1
        ..
25091    0
27853    0
47278    1
37020    1
2217     0
Name: sentiment, Length: 10000, dtype: int64

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [26]:
model =tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(50,dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
train_sentiment.shape

(40000, 10)

In [30]:
x_train=np.array(train_sentiment).reshape((train_sentiment.shape[0],train_sentiment.shape[1],1))

In [31]:
x_test=np.array(test_sentiment).reshape((test_sentiment.shape[0],test_sentiment.shape[1],1))

In [33]:
from keras.preprocessing import sequence

In [67]:
model.fit(x_train,train_review,batch_size=10,epochs=100,validation_data=(x_test,test_review))

Epoch 1/100
4000/4000 [==============================] - 19s 5ms/step - loss: 0.6729 - accuracy: 0.5814 - val_loss: 0.6760 - val_accuracy: 0.5843
Epoch 2/100
4000/4000 [==============================] - 18s 4ms/step - loss: 0.6724 - accuracy: 0.5829 - val_loss: 0.6761 - val_accuracy: 0.5688
Epoch 3/100
4000/4000 [==============================] - 19s 5ms/step - loss: 0.6719 - accuracy: 0.5847 - val_loss: 0.6773 - val_accuracy: 0.5663
Epoch 4/100
4000/4000 [==============================] - 18s 4ms/step - loss: 0.6714 - accuracy: 0.5864 - val_loss: 0.6710 - val_accuracy: 0.5927
Epoch 5/100
4000/4000 [==============================] - 19s 5ms/step - loss: 0.6715 - accuracy: 0.5843 - val_loss: 0.6728 - val_accuracy: 0.5949
Epoch 6/100
4000/4000 [==============================] - 19s 5ms/step - loss: 0.6719 - accuracy: 0.5843 - val_loss: 0.6731 - val_accuracy: 0.5876
Epoch 7/100
4000/4000 [==============================] - 18s 4ms/step - loss: 0.6719 - accuracy: 0.5835 - val_loss: 0.6760 -

In [55]:
from keras.preprocessing.text import Tokenizer

In [59]:
random =[ "The movie was very touching and heart whelming"]

In [60]:
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(random))
test = tf_idf.toarray()

In [61]:
test

array([[0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
        0.35355339, 0.35355339, 0.35355339]])

In [69]:
model.predict(test)

array([[0.49313244]], dtype=float32)